In [24]:
input_folder = "input_imgs"

In [ ]:
!git clone https://github.com/ming053l/DRCT.git
!pip install -r DRCT/requirements.txt
!pip install tqdm

In [26]:
import os
import re
import fileinput
from PIL import Image
import numpy as np
import pandas as pd
import basicsr
import sys


# Directory Setup
output_folder = 'Output_KK'
os.makedirs('Output_data', exist_ok=True)
os.makedirs('Stacked_data', exist_ok=True)

print("Directories created:")
print(os.listdir('.'))


# Function to patch basicsr import issue
def patch_basicsr_import():
    """
    Patch the basicsr 'degradations.py' file to fix import issues.
    """
    try:
        import basicsr
        basicsr_path = os.path.dirname(basicsr.__file__)
        degradations_path = os.path.join(basicsr_path, 'data', 'degradations.py')

        if not os.path.exists(degradations_path):
            print(f"Could not find degradations.py at {degradations_path}")
            return False

        # Create backup if not already created
        backup_path = degradations_path + '.backup'
        if not os.path.exists(backup_path):
            with open(degradations_path, 'r') as src, open(backup_path, 'w') as dst:
                dst.write(src.read())

        # Replace problematic import
        old_import = "from torchvision.transforms.functional_tensor import rgb_to_grayscale"
        new_import = "from torchvision.transforms.functional import rgb_to_grayscale"

        with fileinput.FileInput(degradations_path, inplace=True, backup='.bak') as file:
            for line in file:
                print(line.replace(old_import, new_import), end='')

        print(f"Successfully patched {degradations_path}")
        return True

    except ImportError:
        print("BasicSR package not found. Please ensure it's installed.")
        return False
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False


# Function to patch inference file strict=True issue
def patch_inference_file(file_path="DRCT/inference.py"):
    """
    Modify the inference.py file to replace 'strict=True' with 'strict=False'.
    """
    try:
        # Create backup if not already created
        backup_path = file_path + '.backup'
        if not os.path.exists(backup_path):
            with open(file_path, 'r') as src, open(backup_path, 'w') as dst:
                dst.write(src.read())

        # Replace strict=True with strict=False
        pattern = r'(model\.load_state_dict\(.*strict=)True(\).*)'
        with open(file_path, 'r') as file:
            content = file.read()

        modified_content = re.sub(pattern, r'\1False\2', content)

        with open(file_path, 'w') as file:
            file.write(modified_content)

        print(f"Successfully modified {file_path}")
        print("Changed strict=True to strict=False")
        return True

    except FileNotFoundError:
        print(f"Could not find file at {file_path}")
        return False
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return False


# Execute patch functions
patch_basicsr_import()
patch_inference_file()

# Image Stacking
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff', '.webp')):
        image_path = os.path.join(input_folder, filename)
        image = Image.open(image_path)

        # Resize the image to 32x32
        image = image.resize((32, 32))

        # Create the stacked image (64x64)
        stacked_image = Image.new('RGB', (64, 64))
        stacked_image.paste(image, (0, 0))        # Top-left
        stacked_image.paste(image, (32, 0))      # Top-right
        stacked_image.paste(image, (0, 32))      # Bottom-left
        stacked_image.paste(image, (32, 32))     # Bottom-right

        # Save the stacked image
        base_name, ext = os.path.splitext(filename)
        output_path = os.path.join(output_folder, f"{base_name}_resized{ext}")
        stacked_image.save(output_path)
        print(f"Processed and saved: {output_path}")

print("All eligible images have been processed.")


# Run inference script

!python DRCT/inference.py --model_path Model_weights/net_g_latest.pth --input Output_KK --output Output_data --scale 4

# Cropping Images
input_folder1 = "Output_data"
output_folder = "very_big_img"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder1):
    input_path = os.path.join(input_folder1, filename)

    if os.path.isfile(input_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
        with Image.open(input_path) as img:
            # Define the crop box for the top-left quarter
            width, height = img.size
            crop_box = (0, 0, width // 2, height // 2)
            cropped_img = img.crop(crop_box)

            # Save the cropped image
            output_path = os.path.join(output_folder, filename)
            cropped_img.save(output_path)

print("Cropping complete! Final images are saved in:", output_folder)


Directories created:
['.config', '.local', '.ipython', '.jupyter', '.ipynb_checkpoints', 'Artifact_detection.ipynb', 'Task_2_Pipeline_README.md', 'input_imgs', '.cache', 'Model_weights', 'Artifact_descriptions', '.nv', 'DRCT', 'Output_data', 'Stacked_data']
Successfully patched /root/miniconda3/envs/py3.10/lib/python3.10/site-packages/basicsr/data/degradations.py
Successfully modified DRCT/inference.py
Changed strict=True to strict=False
Processed and saved: Output_KK/1_resized.png
Processed and saved: Output_KK/9_resized.png
Processed and saved: Output_KK/7_resized.png
Processed and saved: Output_KK/3_resized.png
Processed and saved: Output_KK/4_resized.png
Processed and saved: Output_KK/8_resized.png
Processed and saved: Output_KK/10_resized.png
Processed and saved: Output_KK/2_resized.png
Processed and saved: Output_KK/5_resized.png
Processed and saved: Output_KK/6_resized.png
All eligible images have been processed.
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torch/fu

In [27]:
%%capture
!pip install -q transformers==4.36.2
!pip install git+https://github.com/openai/clip
!pip install ftfy
!pip install regex
!pip install tqdm
!pip install pillow
!pip install pytorch
!pip install torchvision
!pip install opencv-contrib-python
!pip install einops
!pip install grad-cam
!pip install open-clip-torch
from transformers import AutoTokenizer, BitsAndBytesConfig
import torch

## Set up for GradCAM model

In [28]:
os.mkdir('testgradcam')
os.chdir('testgradcam')
os.mkdir('test')
os.mkdir('train')
os.chdir('test')
os.mkdir('FAKE')
os.mkdir('REAL')
os.chdir('..')
os.chdir('train')
os.mkdir('FAKE')
os.mkdir('REAL')
os.chdir('..')
os.chdir('..')

In [29]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torchvision.models import resnet50
from pytorch_grad_cam import GradCAM
from pytorch_grad_cam.utils.image import show_cam_on_image
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
import numpy as np
import cv2
from tqdm import tqdm
import os

def load_model(model_path, device='cuda'):
    model = resnet50(pretrained=False)
    model.fc = nn.Linear(model.fc.in_features, 2)
    model.load_state_dict(torch.load(model_path))
    model.to(device)
    model.eval()
    return model
def generate_intensity_matrix(grayscale_cam, size=(32, 32)):
    # Resize the grayscale CAM to desired dimensions
    intensity_matrix = cv2.resize(grayscale_cam, size, interpolation=cv2.INTER_LINEAR)
    # Values are already normalized between 0 and 1 from GradCAM
    return intensity_matrix

def generate_gradcam_visualizations(
    model_path,
    dataset_path,
    output_dir,
    num_samples=1,  # Changed to 1 for testing
    device='cuda'
):
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load model
    model = load_model(model_path, device)

    # Define target layer for GradCAM
    target_layers = [model.layer4[-1]]
    # Initialize GradCAM
    cam = GradCAM(
        model=model,
        target_layers=target_layers,
    )

    # Setup data loading
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                           std=[0.229, 0.224, 0.225])
    ])

    # Load dataset
    dataset = torchvision.datasets.ImageFolder(
        root=dataset_path,
        transform=transform
    )

    # Select random samples
    num_samples = min(num_samples, len(dataset))
    indices = torch.randperm(len(dataset))[:num_samples]

    # Process each image
    for idx in tqdm(indices, desc="Generating Grad-CAM visualizations"):
        # Get image and label
        img_tensor, label = dataset[idx]
        input_tensor = img_tensor.unsqueeze(0)

        # Get original image for overlay
        img_path = dataset.imgs[idx][0]
        rgb_img = cv2.imread(img_path, 1)[:, :, ::-1]
        rgb_img = cv2.resize(rgb_img, (224, 224))
        rgb_img = np.float32(rgb_img) / 255

        # Generate class activation map
        grayscale_cam = cam(input_tensor=input_tensor, targets=None)
        grayscale_cam = grayscale_cam[0, :]

        # Generate 32x32 intensity matrix
        intensity_matrix = generate_intensity_matrix(grayscale_cam)

        # Save intensity matrix
        matrix_output_path = os.path.join(
            output_dir,
            f"intensity_matrix_{0}.npy"
        )
        np.save(matrix_output_path, intensity_matrix)

        # Print shape and some statistics of the intensity matrix
        print(f"Intensity Matrix Shape: {intensity_matrix.shape}")
        print(f"Min value: {intensity_matrix.min():.4f}")
        print(f"Max value: {intensity_matrix.max():.4f}")
        print(f"Mean value: {intensity_matrix.mean():.4f}")

        # Overlay CAM on original image
        visualization = show_cam_on_image(rgb_img, grayscale_cam, use_rgb=True)

        # Get model prediction
        with torch.no_grad():
            output = model(input_tensor.to(device))
            pred = output.argmax(dim=1).item()

        # Save the visualization
        output_path = os.path.join(
            output_dir,
            f"gradcam_pred_true.jpg"
        )
        cv2.imwrite(output_path, visualization[:, :, ::-1])

## Classes for getting Artifact probability using CLIP

In [42]:
import json
import clip
import cv2
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image
import open_clip


# CLIP Model Class - ViT-B/32
class Clip32b:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, self.preprocess = clip.load("ViT-B/32", device=self.device)
        total_params = sum(p.numel() for p in self.model.parameters())
        print("Total params: ", total_params)

    def image_text_search(self, image_patches, text_descriptions):
        """
        Compute text-image similarity for each patch and return scores.
        """
        text = clip.tokenize(text_descriptions).to(self.device)
        patch_results = []

        for patch in image_patches:
            patch = self.preprocess(patch).unsqueeze(0).to(self.device)
            with torch.no_grad():
                logits_per_image, _ = self.model(patch, text)
                probs = logits_per_image.softmax(dim=-1).cpu().numpy()
            patch_results.append(probs[0])

        return np.array(patch_results)

    def split_image_into_patches(self, image, patch_size, intensities):
        """
        Split the image into non-overlapping patches of size (patch_size x patch_size).
        """
        image = np.array(image)
        h, w, c = image.shape
        patches = []
        intensity = []

        for i in range(0, h, patch_size):
            for j in range(0, w, patch_size):
                patch = image[i: i + patch_size, j: j + patch_size]
                gradpatch = intensities[i: i + patch_size, j: j + patch_size]
                if patch.shape[:2] == (patch_size, patch_size):
                    patches.append(Image.fromarray(patch))
                    intensity.append(np.sum(gradpatch))

        return patches, intensity

    def give_result(self, patch_results, patch_weights):
        """
        Perform majority voting to determine the most frequent description.
        """
        votes = np.argmax(patch_results, axis=1)
        if np.sum((votes != 2) * patch_weights) != 0:
            weighted_avg = np.sum((votes == 0) * patch_weights) / np.sum((votes != 2) * patch_weights)
        else:
            weighted_avg = 0
        return weighted_avg

    def final_img_process(self, description_list, image_path, small=1):
        """
        Perform patch-based processing on the image and return weighted average similarity.
        """
        patch_sizes = [32, 64, 128, 256] if small == 1 else [32]
        image = Image.open(image_path)
        intensity = np.load("/gradcam_visualizations/intensity_matrix_0.npy")
        intensity = np.array(Image.fromarray(intensity).resize((128, 128)))

        all_results, all_weights = [], []
        for patch_size in patch_sizes:
            patches, patch_intensity = self.split_image_into_patches(image, patch_size, intensity)
            patch_results = self.image_text_search(patches, description_list)
            all_results.append(list(patch_results))
            all_weights.append(patch_intensity)

        stacked_results = np.array([i for j in all_results for i in j])
        stacked_weights = np.array([i for j in all_weights for i in j])
        return self.give_result(stacked_results, stacked_weights)


# CLIP Model Class - ViT-H/14
class Clip14h:
    def __init__(self):
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model, _, self.preprocess = open_clip.create_model_and_transforms(
            "ViT-H-14-quickgelu", pretrained="dfn5b", device=self.device
        )
        self.tokenizer = open_clip.get_tokenizer("ViT-H-14-quickgelu")
        total_params = sum(p.numel() for p in self.model.parameters())
        print("Total params: ", total_params)

    def image_text_search(self, image_patches, text_descriptions):
        """
        Compute text-image similarity for each patch and return scores.
        """
        text = self.tokenizer(text_descriptions).to(self.device)
        patch_results = []

        for patch in image_patches:
            patch = self.preprocess(patch).unsqueeze(0).to(self.device)
            with torch.no_grad(), torch.cuda.amp.autocast():
                image_features = self.model.encode_image(patch).to(self.device)
                text_features = self.model.encode_text(text).to(self.device)
                image_features /= image_features.norm(dim=-1, keepdim=True)
                text_features /= text_features.norm(dim=-1, keepdim=True)
                probs = (image_features @ text_features.T).softmax(dim=-1).cpu().numpy()
            patch_results.append(probs[0])

        return np.array(patch_results)

    def split_image_into_patches(self, image, patch_size, intensities):
        """
        Split the image into non-overlapping patches of size (patch_size x patch_size).
        """
        image = np.array(image)
        h, w, c = image.shape
        patches = []
        intensity = []

        for i in range(0, h, patch_size):
            for j in range(0, w, patch_size):
                patch = image[i: i + patch_size, j: j + patch_size]
                gradpatch = intensities[i: i + patch_size, j: j + patch_size]
                if patch.shape[:2] == (patch_size, patch_size):
                    patches.append(Image.fromarray(patch))
                    intensity.append(np.sum(gradpatch))

        return patches, intensity

    def give_result(self, patch_results, patch_weights):
        """
        Perform majority voting to determine the most frequent description.
        """
        votes = np.argmax(patch_results, axis=1)
        if np.sum((votes != 2) * patch_weights) != 0:
            weighted_avg = np.sum((votes == 0) * patch_weights) / np.sum((votes != 2) * patch_weights)
        else:
            weighted_avg = 0
        return weighted_avg

    def final_img_process(self, description_list, image_path, small=1):
        """
        Perform patch-based processing on the image and return weighted average similarity.
        """
        patch_sizes = [32, 64, 128, 256] if small == 1 else [32]
        image = Image.open(image_path)
        intensity = np.load("gradcam_visualizations/intensity_matrix_0.npy")
        intensity = np.array(Image.fromarray(intensity).resize((128, 128)))

        all_results, all_weights = [], []
        for patch_size in patch_sizes:
            patches, patch_intensity = self.split_image_into_patches(image, patch_size, intensity)
            patch_results = self.image_text_search(patches, description_list)
            all_results.append(list(patch_results))
            all_weights.append(patch_intensity)

        stacked_results = np.array([i for j in all_results for i in j])
        stacked_weights = np.array([i for j in all_weights for i in j])
        return self.give_result(stacked_results, stacked_weights)


In [43]:
animal_artifacts = [
    "Dental anomalies in mammals",
    "Anatomically incorrect paw structures",
    "Unrealistic eye reflections",
    "Misshapen ears or appendages",
    "Unnatural pose artifacts",
    "Biological asymmetry errors",
    "Impossible foreshortening in animal bodies",
    "Anatomically impossible joint configurations",
    "Misaligned bilateral elements in animal faces",
    "Improper fur direction flows",
    "Incorrect skin tones",
    "Asymmetric features in naturally symmetric objects"
]
vehicle_artifacts = [
    "Impossible mechanical connections",
    "Impossible mechanical joints",
    "Physically impossible structural elements",
    "Incorrect wheel geometry",
    "Implausible aerodynamic structures",
    "Misaligned body panels",
    "Distorted window reflections",
    "Incorrect reflection mapping",
    "Non-manifold geometries in rigid structures",
    "Irregular proportions in mechanical components",
    "Inconsistent scale of mechanical parts",
    "Metallic surface artifacts",
    "Floating or disconnected components",
    "Unnaturally glossy surfaces"
]


In [44]:
import os
import pandas as pd
import torch
from PIL import Image
import json

def find_artifacts_in_image(folder_path, threshold_sr=0.5, threshold_non_sr=0.8):
    sr_artifacts = [
        "Incorrect reflection mapping", "Abruptly cut off objects", "Ghosting effects: Semi-transparent duplicates of elements",
        "Dental anomalies in mammals", "Anatomically incorrect paw structures", "Unrealistic eye reflections",
        "Misshapen ears or appendages", "Unnatural pose artifacts", "Biological asymmetry errors",
        "Impossible foreshortening in animal bodies", "Impossible mechanical connections", "Impossible mechanical joints",
        "Physically impossible structural elements", "Incorrect wheel geometry", "Implausible aerodynamic structures",
        "Misaligned body panels", "Distorted window reflections", "Anatomically impossible joint configurations",
        "Non-manifold geometries in rigid structures", "Asymmetric features in naturally symmetric objects",
        "Misaligned bilateral elements in animal faces", "Irregular proportions in mechanical components",
        "Inconsistent scale of mechanical parts", "Incorrect perspective rendering", "Scale inconsistencies within single objects",
        "Spatial relationship errors", "Scale inconsistencies within the same object class", "Depth perception anomalies"
    ]
    non_sr_artifacts = [
        "Improper fur direction flows", "Incorrect skin tones", "Inconsistent object boundaries",
        "Blurred boundaries in fine details", "Over-sharpening artifacts", "Excessive sharpness in certain image regions",
        "Aliasing along high-contrast edges", "Jagged edges in curved structures", "Fake depth of field",
        "Artificial depth of field in object presentation", "Discontinuous surfaces", "Unnaturally glossy surfaces",
        "Metallic surface artifacts", "Texture bleeding between adjacent regions", "Texture repetition patterns",
        "Over-smoothing of natural textures", "Regular grid-like artifacts in textures", "Artificial noise patterns in uniform surfaces",
        "Random noise patterns in detailed areas", "Repeated element patterns", "Systematic color distribution anomalies",
        "Unnatural color transitions", "Color coherence breaks", "Frequency domain signatures",
        "Artificial smoothness", "Cinematization effects", "Movie-poster-like composition of ordinary scenes",
        "Exaggerated characteristic features", "Synthetic material appearance", "Floating or disconnected components",
        "Inconsistent material properties", "Depth perception anomalies", "Loss of fine detail in complex structures",
        "Resolution inconsistencies within regions", "Abruptly cut off objects", "Unrealistic specular highlights"
    ]

    # Load artifact descriptions
    tuple_desc = pd.read_csv('Artifact_descriptions/Artifact_description_Tuple.csv')

    result = {}
    animal_or_vehicle = Clip32b()
    artifact_detector = Clip14h()
    print("Loaded artifact detector")

    for filename in os.listdir(folder_path):
        if filename.lower().endswith((".png", ".jpg", ".jpeg", ".bmp", ".gif")):
            # Process image
            image_path = os.path.join(folder_path, filename)
            image = Image.open(image_path)
            image.save("testgradcam/test/FAKE/peg32.png")
            image.save("testgradcam/test/REAL/peg32.png")
            image.save("testgradcam/train/FAKE/peg32.png")
            image.save("testgradcam/train/REAL/peg32.png")

            # Generate Grad-CAM visualizations
            generate_gradcam_visualizations(
                model_path='Model_weights/fake_detector.pth',
                dataset_path='testgradcam',
                output_dir='gradcam_visualizations',
                num_samples=1,
                device='cuda' if torch.cuda.is_available() else 'cpu'
            )

            # Determine if the image is an animal or a vehicle
            probab_animal_vehicle = animal_or_vehicle.image_text_search(
                [Image.open(f"very_big_img/{filename[:-4]}_resized_DRCT-L_X4.png")],
                ["Vehicle", "Animal"]
            )[0]
            predicted_class = int(probab_animal_vehicle[0] < probab_animal_vehicle[1])
            if predicted_class:
                print("ANIMAL DETECTED!!!!!!")

            artifacts_present = []

            # Process SR artifacts
            for artifact in sr_artifacts:
                row = tuple_desc[tuple_desc['Artifact'] == artifact]
                if not row.empty:
                    description = row.iloc[0, 1:].tolist()
                else:
                    continue

                if predicted_class == 0 and artifact in animal_artifacts:
                    continue
                if predicted_class == 1 and artifact in vehicle_artifacts:
                    continue

                print(description)
                score = artifact_detector.final_img_process(
                    description, f"very_big_img/{filename[:-4]}_resized_DRCT-L_X4.png"
                )

                if score > threshold_sr:
                    artifacts_present.append(artifact)
                print(filename, "processed for", artifact, "with probability of", score)

            # Process Non-SR artifacts
            for artifact in non_sr_artifacts:
                row = tuple_desc[tuple_desc['Artifact'] == artifact]
                if not row.empty:
                    description = row.iloc[0, 1:].tolist()
                else:
                    continue

                print(description)
                score = artifact_detector.final_img_process(
                    description,  f"very_big_img/{filename[:-4]}_resized_DRCT-L_X4.png"
                )

                if score > threshold_non_sr:
                    artifacts_present.append(artifact)
                print(filename, "processed for", artifact, "with probability of", score)

            result[filename] = artifacts_present

    return result


## Detect Artifacts

In [45]:
results = find_artifacts_in_image(input_folder, 0.85, 0.9)
with open("Artifacts_detected.json", "w") as file:
    json.dump(results, file, indent=4) 

Total params:  151277313
Total params:  986109441
Loaded artifact detector


/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/miniconda3/envs/py3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_103/4231269379.py:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flippe

Intensity Matrix Shape: (32, 32)
Min value: 0.0000
Max value: 0.9577
Mean value: 0.3006
ANIMAL DETECTED!!!!!!
['Cut off object', 'Full object', 'No object']
1.png processed for Abruptly cut off objects with probability of 0.2811575
['Unnatural teeth', 'Natural teeth', 'No teeth']
1.png processed for Dental anomalies in mammals with probability of 1.0
['Unnatural paw shape', 'Natural paw shape', 'No paws']
1.png processed for Anatomically incorrect paw structures with probability of 1.0
['Unnatural ear shape', 'Natural ear shape', 'No ears']
1.png processed for Misshapen ears or appendages with probability of 0.9263857
['Unnatural pose', 'Natural pose', 'Vehicle']
1.png processed for Unnatural pose artifacts with probability of 1.0
['Asymmetric body', 'Symmetric body', 'Vehicle']
1.png processed for Biological asymmetry errors with probability of 0.545261
['Irregular limb size', 'Natural limb size', 'Vehicle']
1.png processed for Impossible foreshortening in animal bodies with probabili

Generating Grad-CAM visualizations: 100%|██████████| 1/1 [00:00<00:00, 36.07it/s]


Intensity Matrix Shape: (32, 32)
Min value: 0.0000
Max value: 0.9801
Mean value: 0.2110
ANIMAL DETECTED!!!!!!
['Cut off object', 'Full object', 'No object']
9.png processed for Abruptly cut off objects with probability of 0.17873985
['Unnatural teeth', 'Natural teeth', 'No teeth']
9.png processed for Dental anomalies in mammals with probability of 0.8329476
['Unnatural paw shape', 'Natural paw shape', 'No paws']
9.png processed for Anatomically incorrect paw structures with probability of 0.7968692
['Unnatural ear shape', 'Natural ear shape', 'No ears']
9.png processed for Misshapen ears or appendages with probability of 0.4145343
['Unnatural pose', 'Natural pose', 'Vehicle']
9.png processed for Unnatural pose artifacts with probability of 1.0
['Asymmetric body', 'Symmetric body', 'Vehicle']
9.png processed for Biological asymmetry errors with probability of 0.5730935
['Irregular limb size', 'Natural limb size', 'Vehicle']
9.png processed for Impossible foreshortening in animal bodies 

...

Intensity Matrix Shape: (32, 32)
Min value: 0.0000
Max value: 0.9638
Mean value: 0.2840
['Unnatural reflections', 'Natural reflections', 'No reflections']
2.png processed for Incorrect reflection mapping with probability of 1.0
['Cut off object', 'Full object', 'No object']
2.png processed for Abruptly cut off objects with probability of 0.1409911
['Incorrect mechanical connection', 'Correct mechanical connection', 'No mechanical connection']
2.png processed for Impossible mechanical connections with probability of 0.84068
['Unnatural mechanical joints', 'Natural mechanical joints', 'No mechanical joints']
2.png processed for Impossible mechanical joints with probability of 0.7258419
['Unreal wheels', 'Round wheels', 'No wheels']
2.png processed for Incorrect wheel geometry with probability of 0.99434114
['Unreal aeroplane', 'Real aeroplane', 'No aeroplane']
2.png processed for Implausible aerodynamic structures with probability of 0.6450372
['Panels not aligned properly', 'Panels alig

Generating Grad-CAM visualizations: 100%|██████████| 1/1 [00:00<00:00, 30.98it/s]


Intensity Matrix Shape: (32, 32)
Min value: 0.0000
Max value: 0.9723
Mean value: 0.2908
['Unnatural reflections', 'Natural reflections', 'No reflections']
5.png processed for Incorrect reflection mapping with probability of 1.0
['Cut off object', 'Full object', 'No object']
5.png processed for Abruptly cut off objects with probability of 0.22252397
['Incorrect mechanical connection', 'Correct mechanical connection', 'No mechanical connection']
5.png processed for Impossible mechanical connections with probability of 0.53578985
['Unnatural mechanical joints', 'Natural mechanical joints', 'No mechanical joints']
5.png processed for Impossible mechanical joints with probability of 0.7785805
['Unreal wheels', 'Round wheels', 'No wheels']
5.png processed for Incorrect wheel geometry with probability of 0.97633684
['Unreal aeroplane', 'Real aeroplane', 'No aeroplane']
5.png processed for Implausible aerodynamic structures with probability of 1.0
['Panels not aligned properly', 'Panels aligne

Generating Grad-CAM visualizations: 100%|██████████| 1/1 [00:00<00:00, 37.78it/s]


Intensity Matrix Shape: (32, 32)
Min value: 0.0000
Max value: 0.9983
Mean value: 0.2071
ANIMAL DETECTED!!!!!!
['Cut off object', 'Full object', 'No object']
6.png processed for Abruptly cut off objects with probability of 0.20642665
['Unnatural teeth', 'Natural teeth', 'No teeth']
6.png processed for Dental anomalies in mammals with probability of 1.0
['Unnatural paw shape', 'Natural paw shape', 'No paws']
6.png processed for Anatomically incorrect paw structures with probability of 1.0
['Unnatural ear shape', 'Natural ear shape', 'No ears']
6.png processed for Misshapen ears or appendages with probability of 0.8862364
['Unnatural pose', 'Natural pose', 'Vehicle']
6.png processed for Unnatural pose artifacts with probability of 1.0
['Asymmetric body', 'Symmetric body', 'Vehicle']
6.png processed for Biological asymmetry errors with probability of 0.859739
['Irregular limb size', 'Natural limb size', 'Vehicle']
6.png processed for Impossible foreshortening in animal bodies with probabil